In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from scipy.signal import butter, filtfilt, lfilter
import matplotlib.pyplot as plt
import gc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.layers import TimeDistributed, Reshape, Permute
from keras.utils import to_categorical
from tensorflow import keras
import pywt
import scipy.signal as signal
from scipy.integrate import simps

import scipy

In [1]:
# ---------- FUNCTIONS ---------- #

#___SCALING___#
def apply_scaling(array):
  array_norm = np.zeros((array.shape[0],array.shape[1]))
  for i in range(array.shape[0]):
    means = np.mean(array[i])  # Calculate mean for each sensor
    stds = np.std(array[i])    # Calculate standard deviation for each sensor
    array_norm[i] = (array[i] - means) / stds   # Subtrack and divide

  del array, means, stds
  gc.collect()
  return array_norm

#___LOWPASS FILTER___#
def butter_lowpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs  # Nyquist Frequency
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def apply_lowpass(array):
  original_sampling_rate = 2034  # Original sampling rate
  downsampling_factor = 4
  new_sampling_rate = original_sampling_rate / downsampling_factor  # New sampling rate after downsampling
  cutoff_frequency = new_sampling_rate / 2  # Nyquist frequency

  array_filtered = np.zeros((array.shape[0], array.shape[1]))

  for i in range(array.shape[0]):  # Iterate over sensors
      array_filtered[i, :] = butter_lowpass_filter(array[i, :], cutoff_frequency, original_sampling_rate)

  del array
  return array_filtered

#___DOWNSAMPLING___#
def apply_downsampling(array):
  n_sensors, n_timepoints = array.shape

  downsampling_factor = 4
  new_n_timepoints = n_timepoints // downsampling_factor
  array_downsamp = np.zeros((n_sensors, new_n_timepoints))

  for sensor in range(n_sensors):
    array_downsamp[sensor,:] = array[sensor, ::downsampling_factor]

  del array, new_n_timepoints, downsampling_factor, n_sensors, n_timepoints
  gc.collect()
  return array_downsamp




In [4]:
# LOAD FILES

def get_file_paths_and_labels(data_folder, task_numbers):
  file_paths = []
  labels = []
  for task_number in task_numbers:
      for file in os.listdir(data_folder):
          if file.endswith(task_number + 'h5'):
              file_path = os.path.join(data_folder, file)
              file_paths.append(file_path)
              labels.append(assign_label(file))
  return file_paths, labels

def find_fmri_data_folder(start_path):
    for root, dirs, files in os.walk(start_path):
        if 'meg_data' in dirs:
            return os.path.join(root, 'meg_data/Intra/train')
    raise Exception("meg_data folder not found. Please check the directory structure.")

def load_data(file_path):
  with h5py.File(file_path, 'r') as f:
      dataset_name = get_dataset_name(file_path)
      matrix = f.get(dataset_name)[:]
      return matrix

def get_dataset_name(file_name_with_dir):
  filename_without_dir = file_name_with_dir.split('/')[-1]
  temp = filename_without_dir.split('_')[:-1]
  dataset_name = "_".join(temp)
  return dataset_name

def assign_label(file_name):
  if file_name.startswith("rest"):
      return 0
  elif file_name.startswith("task_motor"):
      return 1
  elif file_name.startswith("task_story"):
      return 2
  elif file_name.startswith("task_working"):
      return 3
  else:
      return None

def count_files_with_task_numbers(data_folder, task_numbers):
    total_files = 0
    for file in os.listdir(data_folder):
        if any(file.endswith(task_number + 'h5') for task_number in task_numbers):
            total_files += 1
    return total_files

In [5]:
# Call Preprocessing functions
def preprocess_data(data, i):
  print(f"*** FILE {i} ***")
  data = data[:, :-8]
  data = apply_scaling(data)
  print("scaling applied: shape:", data.shape, end=', ')
  data = apply_lowpass(data)
  print("lowpass applied: shape:", data.shape, end=', ')
  data = apply_downsampling(data)
  print("downsam applied: shape:", data.shape, end=', ')

  return np.array(data)


In [6]:
# Retieve the initial data from the folder

fmri_data_folder = find_fmri_data_folder('/content/drive')
print("fmri_data_folder:", fmri_data_folder)

meg_data_list = []
labels = []

for file in os.listdir(fmri_data_folder):
    if file.endswith('.h5'):
        file_path = os.path.join(fmri_data_folder, file)
        data = load_data(file_path)
        meg_data_list.append(data)
        labels.append(assign_label(file))

        # Clear memory
        del data
        gc.collect()

# Convert the list of 2D arrays into a single 3D NumPy array
meg_train_data_array = np.stack(meg_data_list, axis=0)
labels_train_array = np.array(labels)

del meg_data_list, labels

fmri_data_folder: /content/drive/MyDrive/Courses/Pattern Recognition/Lab/Group Assignment/meg_data/Intra/train


In [7]:
print("## Initial shapes of the data ##")
print("MEG:", meg_train_data_array.shape)
print("Labels:",labels_train_array.shape)

## Initial shapes of the data ##
MEG: (32, 248, 35624)
Labels: (32,)


In [9]:
# ----- CALL PROCESSING FUNCTIONS FOR ALL FILES ----- #

processed_data = np.zeros((meg_train_data_array.shape[0], meg_train_data_array.shape[1], 8904))
print(processed_data.shape)
for i in range(meg_train_data_array.shape[0]):
  processed_data[i] = preprocess_data(meg_train_data_array[i], i+1)

(32, 248, 8904)
*** FILE 1 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616), downsam applied: shape: (248, 8904), *** FILE 2 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616), downsam applied: shape: (248, 8904), *** FILE 3 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616), downsam applied: shape: (248, 8904), *** FILE 4 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616), downsam applied: shape: (248, 8904), *** FILE 5 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616), downsam applied: shape: (248, 8904), *** FILE 6 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616), downsam applied: shape: (248, 8904), *** FILE 7 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616), downsam applied: shape: (248, 8904), *** FILE 8 ***
scaling applied: shape: (248, 35616), lowpass applied: shape: (248, 35616

In [11]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 43.1 MB/s eta 0:00:00


In [17]:
import mne
from mne.io import RawArray
from mne.channels import make_standard_montage

# Define the sampling frequency and channel information
sfreq = 2034  # Replace with the actual sampling frequency of your MEG device
ch_names = [f'MEG{i}' for i in range(1, 249)]  # Adjust the names based on your device
ch_types = ['mag'] * 248  # Assuming all channels are magnetometers; adjust if needed

# Create MNE info structure
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

# Convert data to Volts (if necessary) and transpose to match MNE's expected format
data_mne = processed_data.transpose(1, 2, 0)  # Now shape is (248, 8904, 32)
print(data_mne.shape)
# Creating the Raw object
raw = RawArray(data_mne.reshape(248, -1), info)

print(len(raw))
# Plotting the raw data
# raw.plot()


(248, 8904, 32)
Creating RawArray with float64 data, n_channels=248, n_times=284928
    Range : 0 ... 284927 =      0.000 ...   140.082 secs
Ready.
284928


In [20]:
import numpy as np
import mne
from mne.viz import plot_topomap

# Assuming initial data shape is (32 observations, 248 sensors, 8904 timepoints)
# Example data - replace with actual data

# Segment the data into 5 segments and calculate the mean of each segment
num_segments = 4
segment_length = processed_data.shape[2] // num_segments
segmented_data = np.mean(processed_data.reshape(processed_data.shape[0], processed_data.shape[1], num_segments, segment_length), axis=3)



In [21]:
segmented_data.shape

(32, 248, 4)

In [30]:
first_observation = segmented_data[0, :, 0]  # Shape: (248,)

# Create MNE info structure
ch_names = [f"MEG{i}" for i in range(1, 249)]  # Generate channel names
ch_types = ['mag'] * 248  # Assuming all channels are magnetometers
sfreq = 1000  # Sample frequency in Hz
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)

# Create MNE RawArray
raw = mne.io.RawArray(first_observation[np.newaxis, :], info)

# Plotting the sensor locations on a head map
# Note: Without actual sensor positions, this will use a default layout
raw.plot_sensors(show_names=True, show=True)

ValueError: len(data) (1) does not match len(info["ch_names"]) (248)